Importing NFL Reciever Data

Data Sources
    - (NOT SURE YOU CAN PULL FROM HERE) https://nextgenstats.nfl.com/stats/receiving#percent-intended-yards
    - https://fantasydata.com/nfl/fantasy-football-leaders?season=2023&seasontype=1&scope=1&subscope=1&scoringsystem=2&aggregatescope=1&range=3
    - https://premium.pff.com/nfl/positions/2023/REGPO/receiving?position=WR,TE,RB,ORCV
    - https://nflsavant.com/targets.php?ddlYear=2022&ddlPosition=&week=&rz=redzone&ddlTeam=
    - https://www.pro-football-reference.com/years/2023/fantasy.htm
    - https://www.pro-football-reference.com/years/2023/
    - https://premium.pff.com/nfl/positions/2023/REG/ol-pass-blocking-efficiency
    - 

Final Data Sources
    - pff_receiving: https://premium.pff.com/nfl/positions/2023/REG/receiving-scheme?minimum=1
    - pff_man_zone: https://premium.pff.com/nfl/positions/2023/REG/receiving
    - pfr_fp: https://www.pro-football-reference.com/years/2023/fantasy.htm
    - pfr_rz: https://www.pro-football-reference.com/years/2023/redzone-receiving.htm
    - pfr_tm_off: https://www.pro-football-reference.com/years/2023/ (Team Offense Table)
    - pff_ol: https://premium.pff.com/nfl/positions/2023/REG/ol-pass-blocking-efficiency
    - pff_qb: https://premium.pff.com/nfl/positions/2023/REG/passing?position=QB
    - qb_depth_chart: https://www.ourlads.com/nfldepthcharts/

Problems
1. Rookies
2. Injuries
    - Could do most previous non injured season
    - Need to do everything per game
3. New teams
4. New coaches/schemes

Model Types
 - Linear Regression
 - H20 regression
 - Random Forest
 - Decision Tree
 - Gradient Boost
 - Neural Network
 - Bayesian Regression
 - Ensemble Methods (Combine multiple others)

** Overall Ideas **
- Maybe it would be interesting to value players by projected points above average (of position) and then have the AI do mock drafts to look at the strategy?


**Variable Ideas**

- Last year personal stats:
    - Fantasy Points per game (PFR [pfr_fantasy])
    - Target Share
        - Need to somehow make this per game
        - Maybe just do Targets per game (PFF [pff_receiving])
        - Redzone Targets per game (pfr [pfr_rz])
    - Yards/Game (PFF [pff_receiving])
    - Catch Rate (PFF [pff_receiving])
    - Yards after catch (average) (PFF [pff_receiving])
    - TD's (Expected TD's) (per game) (PFF [pff_receiving])
    - Efficiency against Man (PFF [pff_man_zone])
    - Efficiency against Zone (PFF [pff_man_zone])
    - Average Seperation/Cushion? (Next Gen Stats Don't think you can download)
    - Average Depth of Target (PFF [pff_receiving])
    - % Slot/Outside (PFF [pff_receiving])
    - Number/% of Snaps (PFF [pff_receiving])
    - Position in Depth Chart
    - RECV (PFF [pff_receiving])
    - Yards / Routes Run (PFF [pff_receiving])
    - Routes Run per game


- Last year (new) team stats:
    - Offense rating (PFR [pfr_tm_off])
    - Pass Volume (PFR [pfr_tm_off])
    - QB Ranking (PFF [pff_qb])
    - O-Line Pass Block Ranking (PFF [pff_ol])
    - Time of Possession

- This year personal stats:
    - Age/Experience (Age is in PFR [pfr_fp])
    - Position in Depth Chart

- Overall Status
    - % increase/decrease in PPG from last year (Need to calculate)

- This year team stats:
    - Vacant Target Share



Reading in pff_receiving data

In [1]:
import pandas as pd

# Reading in pff_receiving data

pr14_15 = pd.read_csv('WR_Data/pff_receiving_14-15.csv')
pr15_16 = pd.read_csv('WR_Data/pff_receiving_15-16.csv')
pr16_17 = pd.read_csv('WR_Data/pff_receiving_16-17.csv')
pr17_18 = pd.read_csv('WR_Data/pff_receiving_17-18.csv')
pr18_19 = pd.read_csv('WR_Data/pff_receiving_18-19.csv')
pr19_20 = pd.read_csv('WR_Data/pff_receiving_19-20.csv')
pr20_21 = pd.read_csv('WR_Data/pff_receiving_20-21.csv')
pr21_22 = pd.read_csv('WR_Data/pff_receiving_21-22.csv')
pr22_23 = pd.read_csv('WR_Data/pff_receiving_22-23.csv')
pr23_24 = pd.read_csv('WR_Data/pff_receiving_23-24.csv')

# Creating a season column to differentiate between seasons

pr14_15['Season'] = '2014-15'
pr15_16['Season'] = '2015-16'
pr16_17['Season'] = '2016-17'
pr17_18['Season'] = '2017-18'
pr18_19['Season'] = '2018-19'
pr19_20['Season'] = '2019-20'
pr20_21['Season'] = '2020-21'
pr21_22['Season'] = '2021-22'
pr22_23['Season'] = '2022-23'
pr23_24['Season'] = '2023-24'

# Combining all of the different season's data into one pff_receiving dataset

pff_receiving = pd.concat([pr14_15, pr15_16, pr16_17, pr17_18, pr18_19, pr19_20, pr20_21, pr21_22, pr22_23, pr23_24])

# Creating PPR PPG column to calculate fantasy points for a season. NEED TO CORRECT WITH ALL INFO NOT JUST REC

pff_receiving['PPR PPG'] = (pff_receiving['touchdowns'] * 6 + (pff_receiving['yards'] / 10) + pff_receiving['receptions']) / pff_receiving['player_game_count']

# Filter on only Wide Recievers

pff_receiving = pff_receiving[pff_receiving['position'] == 'WR']

pff_receiving

,player,player_id,position,team_name,player_game_count,avg_depth_of_target,avoided_tackles,caught_percent,contested_catch_rate,contested_receptions,...,touchdowns,wide_rate,wide_snaps,yards,yards_after_catch,yards_after_catch_per_reception,yards_per_reception,yprr,Season,PPR PPG
0,Antonio Brown,5718,WR,PIT,16,10.4,17,72.5,NaN,0,...,13,82.1,552,1698,615,4.8,13.2,2.67,2014-15,23.550000
1,Demaryius Thomas,5547,WR,DEN,16,10.5,11,62.4,NaN,0,...,11,80.7,505,1619,668,6.0,14.6,2.78,2014-15,21.181250
2,Julio Jones,6158,WR,ATL,15,12.4,16,66.7,NaN,0,...,6,86.7,534,1593,553,5.3,15.3,2.72,2014-15,19.953333
3,Jordy Nelson,4350,WR,GB,16,12.0,14,67.1,NaN,0,...,13,79.2,480,1519,519,5.3,15.5,2.66,2014-15,20.493750
4,Kelvin Benjamin,8663,WR,CAR,16,14.9,9,51.4,NaN,0,...,9,87.4,500,1008,177,2.4,13.8,1.86,2014-15,14.237500
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
458,Trent Taylor,11932,WR,CHI,13,4.0,0,0.0,0.0,0,...,0,36.9,24,0,0,NaN,NaN,0.00,2023-24,0.000000
466,Kwamie Lassiter II,23226,WR,CIN,1,-2.0,0,100.0,NaN,0,...,0,100.0,2,2,4,4.0,2.0,1.00,2023-24,1.200000
475,Shedrick Jackson,84136,WR,CIN,1,31.0,0,0.0,0.0,0,...,0,66.7,2,0,0,NaN,NaN,0.00,2023-24,0.000000
476,Collin Johnson,47808,WR,CHI,3,7.0,0,100.0,NaN,0,...,0,60.0,9,11,4,4.0,11.0,0.79,2023-24,0.700000


Reading in the pff_man_zone data

In [2]:
pmz14_15 = pd.read_csv('WR_Data/pff_man_zone_14-15.csv')
pmz15_16 = pd.read_csv('WR_Data/pff_man_zone_15-16.csv')
pmz16_17 = pd.read_csv('WR_Data/pff_man_zone_16-17.csv')
pmz17_18 = pd.read_csv('WR_Data/pff_man_zone_17-18.csv')
pmz18_19 = pd.read_csv('WR_Data/pff_man_zone_18-19.csv')
pmz19_20 = pd.read_csv('WR_Data/pff_man_zone_19-20.csv')
pmz20_21 = pd.read_csv('WR_Data/pff_man_zone_20-21.csv')
pmz21_22 = pd.read_csv('WR_Data/pff_man_zone_21-22.csv')
pmz22_23 = pd.read_csv('WR_Data/pff_man_zone_22-23.csv')
pmz23_24 = pd.read_csv('WR_Data/pff_man_zone_23-24.csv')

pmz14_15['Season'] = '2014-15'
pmz15_16['Season'] = '2015-16'
pmz16_17['Season'] = '2016-17'
pmz17_18['Season'] = '2017-18'
pmz18_19['Season'] = '2018-19'
pmz19_20['Season'] = '2019-20'
pmz20_21['Season'] = '2020-21'
pmz21_22['Season'] = '2021-22'
pmz22_23['Season'] = '2022-23'
pmz23_24['Season'] = '2023-24'

pff_man_zone = pd.concat([pmz14_15, pmz15_16, pmz16_17, pmz17_18, pmz18_19, pmz19_20, pmz20_21, pmz21_22, pmz22_23, pmz23_24])

pff_man_zone = pff_man_zone[pff_man_zone['position'] == 'WR']

pff_man_zone




,player,player_id,position,team_name,player_game_count,base_targets,declined_penalties,franchise_id,man_avg_depth_of_target,man_avoided_tackles,...,zone_targeted_qb_rating,zone_targets,zone_targets_percent,zone_touchdowns,zone_yards,zone_yards_after_catch,zone_yards_after_catch_per_reception,zone_yards_per_reception,zone_yprr,Season
0,Antonio Brown,5718,WR,PIT,16,178,0,25,10.4,10,...,98.0,101,25.8,1,964,335,4.5,13.0,2.46,2014-15
1,Demaryius Thomas,5547,WR,DEN,16,178,0,10,10.0,5,...,100.2,102,28.0,4,1023,379,5.6,15.0,2.81,2014-15
2,Julio Jones,6158,WR,ATL,15,156,0,2,15.0,4,...,101.0,109,25.6,3,1107,386,5.0,14.4,2.60,2014-15
3,Jordy Nelson,4350,WR,GB,16,146,0,12,11.0,6,...,124.1,86,24.5,6,938,274,4.6,15.6,2.67,2014-15
4,Kelvin Benjamin,8663,WR,CAR,16,142,0,5,12.5,2,...,87.5,88,26.0,5,703,109,2.4,15.6,2.07,2014-15
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
458,Trent Taylor,11932,WR,CHI,13,1,0,6,NaN,0,...,39.6,1,2.7,0,0,0,NaN,NaN,0.00,2023-24
466,Kwamie Lassiter II,23226,WR,CIN,1,1,0,7,-2.0,0,...,0.0,0,0.0,0,0,0,NaN,NaN,0.00,2023-24
475,Shedrick Jackson,84136,WR,CIN,1,1,0,7,NaN,0,...,39.6,1,50.0,0,0,0,NaN,NaN,0.00,2023-24
476,Collin Johnson,47808,WR,CHI,3,1,0,6,NaN,0,...,112.5,1,20.0,0,11,4,4.0,11.0,2.20,2023-24


Reading in pfr_rz data

In [3]:
pfr_rz14_15 = pd.read_csv('WR_Data/pfr_rz_14-15.csv')
pfr_rz15_16 = pd.read_csv('WR_Data/pfr_rz_15-16.csv')
pfr_rz16_17 = pd.read_csv('WR_Data/pfr_rz_16-17.csv')
pfr_rz17_18 = pd.read_csv('WR_Data/pfr_rz_17-18.csv')
pfr_rz18_19 = pd.read_csv('WR_Data/pfr_rz_18-19.csv')
pfr_rz19_20 = pd.read_csv('WR_Data/pfr_rz_19-20.csv')
pfr_rz20_21 = pd.read_csv('WR_Data/pfr_rz_20-21.csv')
pfr_rz21_22 = pd.read_csv('WR_Data/pfr_rz_21-22.csv')
pfr_rz22_23 = pd.read_csv('WR_Data/pfr_rz_22-23.csv')
pfr_rz23_24 = pd.read_csv('WR_Data/pfr_rz_23-24.csv')

pfr_rz14_15['Season'] = '2014-15'
pfr_rz15_16['Season'] = '2015-16'
pfr_rz16_17['Season'] = '2016-17'
pfr_rz17_18['Season'] = '2017-18'
pfr_rz18_19['Season'] = '2018-19'
pfr_rz19_20['Season'] = '2019-20'
pfr_rz20_21['Season'] = '2020-21'
pfr_rz21_22['Season'] = '2021-22'
pfr_rz22_23['Season'] = '2022-23'
pfr_rz23_24['Season'] = '2023-24'

pfr_rz = pd.concat([pfr_rz14_15, pfr_rz15_16, pfr_rz16_17, pfr_rz17_18, pfr_rz18_19, pfr_rz19_20, pfr_rz20_21, pfr_rz21_22, pfr_rz22_23, pfr_rz23_24])

pfr_rz

,Player,Tm,Tgt20,Rec20,Ctch%20,Yds20,TD20,%Tgt20,Tgt10,Rec10,Ctch%10,Yds10,TD10,%Tgt10,Link,Season
0,Demaryius Thomas,DEN,39,18,46.15%,137,6,36.80%,18,6,33.33%,33,4,33.30%,All Demaryius Thomas red zone receiving plays,2014-15
1,Antonio Brown,PIT,33,18,54.55%,133,8,36.70%,18,9,50.00%,45,6,36.00%,All Antonio Brown red zone receiving plays,2014-15
2,Andre Johnson,HOU,26,12,46.15%,54,3,38.80%,11,5,45.45%,25,3,37.90%,All Andre Johnson red zone receiving plays,2014-15
3,Odell Beckham,NYG,25,16,64.00%,122,8,25.50%,10,6,60.00%,42,6,19.20%,All Odell Beckham red zone receiving plays,2014-15
4,Randall Cobb,GNB,25,16,64.00%,94,10,26.00%,14,10,71.43%,37,8,23.70%,All Randall Cobb red zone receiving plays,2014-15
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
344,Jamaal Williams,NOR,1,1,100.00%,3,0,1.50%,0,0,NaN,0,0,0.00%,All Jamaal Williams red zone receiving plays,2023-24
345,Trayveon Williams,CIN,1,1,100.00%,3,0,1.40%,0,0,NaN,0,0,0.00%,All Trayveon Williams red zone receiving plays,2023-24
346,Jeff Wilson,MIA,1,0,0.00%,0,0,1.50%,1,0,0.00%,0,0,2.30%,All Jeff Wilson red zone receiving plays,2023-24
347,Charlie Woerner,SFO,1,1,100.00%,3,0,1.40%,0,0,NaN,0,0,0.00%,All Charlie Woerner red zone receiving plays,2023-24


Reading in pfr_fp data

In [4]:
pfr_fp14_15 = pd.read_csv('WR_Data/pfr_fp_14-15.csv')
pfr_fp15_16 = pd.read_csv('WR_Data/pfr_fp_15-16.csv')
pfr_fp16_17 = pd.read_csv('WR_Data/pfr_fp_16-17.csv')
pfr_fp17_18 = pd.read_csv('WR_Data/pfr_fp_17-18.csv')
pfr_fp18_19 = pd.read_csv('WR_Data/pfr_fp_18-19.csv')
pfr_fp19_20 = pd.read_csv('WR_Data/pfr_fp_19-20.csv')
pfr_fp20_21 = pd.read_csv('WR_Data/pfr_fp_20-21.csv')
pfr_fp21_22 = pd.read_csv('WR_Data/pfr_fp_21-22.csv')
pfr_fp22_23 = pd.read_csv('WR_Data/pfr_fp_22-23.csv')
pfr_fp23_24 = pd.read_csv('WR_Data/pfr_fp_23-24.csv')

pfr_fp14_15['Season'] = '2014-15'
pfr_fp15_16['Season'] = '2015-16'
pfr_fp16_17['Season'] = '2016-17'
pfr_fp17_18['Season'] = '2017-18'
pfr_fp18_19['Season'] = '2018-19'
pfr_fp19_20['Season'] = '2019-20'
pfr_fp20_21['Season'] = '2020-21'
pfr_fp21_22['Season'] = '2021-22'
pfr_fp22_23['Season'] = '2022-23'
pfr_fp23_24['Season'] = '2023-24'

pfr_fp = pd.concat([pfr_fp14_15, pfr_fp15_16, pfr_fp16_17, pfr_fp17_18, pfr_fp18_19, pfr_fp19_20, pfr_fp20_21, pfr_fp21_22, pfr_fp22_23, pfr_fp23_24])

pfr_fp['Player'] = pfr_fp['Player'].str.replace('[*+]', '', regex=True)

pfr_fp = pfr_fp[pfr_fp['FantPos'] == 'WR']

Reading in pfr_tm_off data

In [5]:
pfr_tm_off_14_15 = pd.read_csv('WR_Data/pfr_tm_off_14-15.csv')
pfr_tm_off_15_16 = pd.read_csv('WR_Data/pfr_tm_off_15-16.csv')
pfr_tm_off_16_17 = pd.read_csv('WR_Data/pfr_tm_off_16-17.csv')
pfr_tm_off_17_18 = pd.read_csv('WR_Data/pfr_tm_off_17-18.csv')
pfr_tm_off_18_19 = pd.read_csv('WR_Data/pfr_tm_off_18-19.csv')
pfr_tm_off_19_20 = pd.read_csv('WR_Data/pfr_tm_off_19-20.csv')
pfr_tm_off_20_21 = pd.read_csv('WR_Data/pfr_tm_off_20-21.csv')
pfr_tm_off_21_22 = pd.read_csv('WR_Data/pfr_tm_off_21-22.csv')
pfr_tm_off_22_23 = pd.read_csv('WR_Data/pfr_tm_off_22-23.csv')
pfr_tm_off_23_24 = pd.read_csv('WR_Data/pfr_tm_off_23-24.csv')

pfr_tm_off_14_15['Season'] = '2014-15'
pfr_tm_off_15_16['Season'] = '2015-16'
pfr_tm_off_16_17['Season'] = '2016-17'
pfr_tm_off_17_18['Season'] = '2017-18'
pfr_tm_off_18_19['Season'] = '2018-19'
pfr_tm_off_19_20['Season'] = '2019-20'
pfr_tm_off_20_21['Season'] = '2020-21'
pfr_tm_off_21_22['Season'] = '2021-22'
pfr_tm_off_22_23['Season'] = '2022-23'
pfr_tm_off_23_24['Season'] = '2023-24'

pfr_tm_off = pd.concat([pfr_tm_off_14_15, pfr_tm_off_15_16, pfr_tm_off_16_17, pfr_tm_off_17_18, pfr_tm_off_18_19, pfr_tm_off_19_20, pfr_tm_off_20_21, pfr_tm_off_21_22, pfr_tm_off_22_23, pfr_tm_off_23_24])

pfr_tm_off = pfr_tm_off[~pfr_tm_off['Tm'].isin(['Avg Team', 'League Total', 'Avg Tm/G'])]

pfr_tm_off = pfr_tm_off.rename(columns={"G":"Team_G"})

pfr_tm_off

,Rk,Tm,Team_G,PF,Tot_Yds,Tot_Ply,Y/P,TO,FL,1stD,...,R_TD,R_Y/A,R_1stD,Pen,Yds,1stPy,Sc%,TO%,EXP,Season
0,1.0,Green Bay Packers,16.0,486.0,6178.0,1001.0,6.2,13.0,7.0,356.0,...,14.0,4.4,104.0,92.0,775.0,30.0,46.7,7.1,172.28,2014-15
1,2.0,Denver Broncos,16.0,482.0,6446.0,1067.0,6.0,20.0,5.0,360.0,...,15.0,4.0,98.0,120.0,1045.0,35.0,39.9,9.6,140.40,2014-15
2,3.0,Philadelphia Eagles,16.0,474.0,6348.0,1127.0,5.6,36.0,15.0,356.0,...,16.0,4.2,109.0,115.0,995.0,29.0,36.9,17.2,35.17,2014-15
3,4.0,New England Patriots,16.0,468.0,5848.0,1073.0,5.5,13.0,4.0,361.0,...,13.0,3.9,96.0,120.0,1080.0,37.0,45.3,7.2,181.12,2014-15
4,5.0,Dallas Cowboys,16.0,467.0,6138.0,1014.0,6.1,25.0,14.0,342.0,...,16.0,4.6,106.0,105.0,839.0,31.0,43.6,12.8,174.23,2014-15
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
27,28.0,Pittsburgh Steelers,17.0,304.0,5173.0,1029.0,5.0,16.0,7.0,287.0,...,16.0,4.1,115.0,86.0,753.0,19.0,29.9,7.7,-16.68,2023-24
28,29.0,New York Jets,17.0,268.0,4566.0,1053.0,4.3,33.0,18.0,260.0,...,7.0,4.2,77.0,124.0,945.0,27.0,26.0,13.9,-199.77,2023-24
29,30.0,New York Giants,17.0,266.0,4760.0,1057.0,4.5,19.0,7.0,267.0,...,10.0,4.1,97.0,89.0,711.0,29.0,23.6,8.5,-130.06,2023-24
30,31.0,New England Patriots,17.0,236.0,4696.0,1020.0,4.6,29.0,8.0,261.0,...,9.0,3.9,93.0,89.0,676.0,15.0,20.2,14.3,-157.57,2023-24


Reading in pff_qb data, and the qb_depth_chart data. Joining them together to make pff_qb_starters.

In [6]:
pff_qb_14_15 = pd.read_csv('WR_Data/pff_qb_14-15.csv')
pff_qb_15_16 = pd.read_csv('WR_Data/pff_qb_15-16.csv')
pff_qb_16_17 = pd.read_csv('WR_Data/pff_qb_16-17.csv')
pff_qb_17_18 = pd.read_csv('WR_Data/pff_qb_17-18.csv')
pff_qb_18_19 = pd.read_csv('WR_Data/pff_qb_18-19.csv')
pff_qb_19_20 = pd.read_csv('WR_Data/pff_qb_19-20.csv')
pff_qb_20_21 = pd.read_csv('WR_Data/pff_qb_20-21.csv')
pff_qb_21_22 = pd.read_csv('WR_Data/pff_qb_21-22.csv')
pff_qb_22_23 = pd.read_csv('WR_Data/pff_qb_22-23.csv')
pff_qb_23_24 = pd.read_csv('WR_Data/pff_qb_23-24.csv')

pff_qb_14_15['Season'] = '2014-15'
pff_qb_15_16['Season'] = '2015-16'
pff_qb_16_17['Season'] = '2016-17'
pff_qb_17_18['Season'] = '2017-18'
pff_qb_18_19['Season'] = '2018-19'
pff_qb_19_20['Season'] = '2019-20'
pff_qb_20_21['Season'] = '2020-21'
pff_qb_21_22['Season'] = '2021-22'
pff_qb_22_23['Season'] = '2022-23'
pff_qb_23_24['Season'] = '2023-24'

pff_qb = pd.concat([pff_qb_14_15, pff_qb_15_16, pff_qb_16_17, pff_qb_17_18, pff_qb_18_19, pff_qb_19_20, pff_qb_20_21, pff_qb_21_22, pff_qb_22_23, pff_qb_23_24])

pff_qb = pff_qb[pff_qb['position'] == 'QB']

# Make a column for lowercase player name to join with our depth chart scraped data

pff_qb['lower_name'] = pff_qb['player'].apply(lambda x: x.lower() if isinstance(x, str) else x)

# Rename the player_id column to qb_id

pff_qb = pff_qb.rename(columns={"player_id": "qb_id", "player":"qb"})

# Read in qb_depth_chart data

qb_depth_chart = pd.read_csv('qb_depth_chart.csv')

# Merge pff_qb with qb_depth_chart

pff_qb_starters = pd.merge(left=qb_depth_chart, right=pff_qb, how='left', left_on=['Season', 'Name'], right_on=['Season', 'lower_name'])

# Need to use qb_depth_chart's Team for merging, not pff_qb's

pff_qb = pff_qb.drop(columns=['team_name'])
pff_qb_starters = pff_qb_starters.drop(columns=['team_name'])

# Change team names in Depth Chart data to the standard 
depth_chart_key = {'ARZ':'ARI'}

pff_qb_starters['Team'] = pff_qb_starters['Team'].replace(depth_chart_key)


pff_qb 




,qb,qb_id,position,player_game_count,accuracy_percent,aimed_passes,attempts,avg_depth_of_target,avg_time_to_throw,bats,...,scrambles,spikes,thrown_aways,touchdowns,turnover_worthy_plays,twp_rate,yards,ypa,Season,lower_name
0,Drew Brees,802,QB,16,80.2,607,659,8.1,2.61,13,...,14,6,24,33,28,3.8,4952,7.5,2014-15,drew brees
1,Matt Ryan,4317,QB,16,76.4,588,628,8.2,2.60,8,...,24,5,19,28,26,3.6,4694,7.5,2014-15,matt ryan
2,Andrew Luck,7007,QB,16,73.3,570,616,9.2,2.83,14,...,37,0,22,40,31,4.3,4762,7.7,2014-15,andrew luck
3,Ben Roethlisberger,1732,QB,16,75.3,580,608,9.1,2.47,8,...,7,9,6,32,16,2.3,4963,8.2,2014-15,ben roethlisberger
4,Matthew Stafford,4924,QB,16,71.9,555,602,8.3,2.64,9,...,17,3,28,22,17,2.4,4247,7.1,2014-15,matthew stafford
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
73,Malik Willis,60341,QB,1,80.0,5,5,7.6,3.55,0,...,3,0,0,0,0,0.0,74,14.8,2023-24,malik willis
74,Logan Woodside,46517,QB,1,75.0,4,4,2.0,2.55,0,...,1,0,0,0,1,16.7,27,6.8,2023-24,logan woodside
81,Aaron Rodgers,2241,QB,1,NaN,0,1,NaN,4.15,0,...,0,0,1,0,0,0.0,0,0.0,2023-24,aaron rodgers
97,Sean Clifford,51708,QB,1,100.0,1,1,33.0,2.60,0,...,0,0,0,0,0,0.0,37,37.0,2023-24,sean clifford


Reading in pff_ol data

In [7]:
pff_ol_14_15 = pd.read_csv('WR_Data/pff_ol_14-15.csv')
pff_ol_15_16 = pd.read_csv('WR_Data/pff_ol_15-16.csv')
pff_ol_16_17 = pd.read_csv('WR_Data/pff_ol_16-17.csv')
pff_ol_17_18 = pd.read_csv('WR_Data/pff_ol_17-18.csv')
pff_ol_18_19 = pd.read_csv('WR_Data/pff_ol_18-19.csv')
pff_ol_19_20 = pd.read_csv('WR_Data/pff_ol_19-20.csv')
pff_ol_20_21 = pd.read_csv('WR_Data/pff_ol_20-21.csv')
pff_ol_21_22 = pd.read_csv('WR_Data/pff_ol_21-22.csv')
pff_ol_22_23 = pd.read_csv('WR_Data/pff_ol_22-23.csv')
pff_ol_23_24 = pd.read_csv('WR_Data/pff_ol_23-24.csv')

pff_ol_14_15['Season'] = '2014-15'
pff_ol_15_16['Season'] = '2015-16'
pff_ol_16_17['Season'] = '2016-17'
pff_ol_17_18['Season'] = '2017-18'
pff_ol_18_19['Season'] = '2018-19'
pff_ol_19_20['Season'] = '2019-20'
pff_ol_20_21['Season'] = '2020-21'
pff_ol_21_22['Season'] = '2021-22'
pff_ol_22_23['Season'] = '2022-23'
pff_ol_23_24['Season'] = '2023-24'

pff_ol = pd.concat([pff_ol_14_15, pff_ol_15_16, pff_ol_16_17, pff_ol_17_18, pff_ol_18_19, pff_ol_19_20, pff_ol_20_21, pff_ol_21_22, pff_ol_22_23, pff_ol_23_24])

pff_ol

,team_name,franchise_id,player_game_count,attempts,hits_allowed,hurries_allowed,pass_snaps,pbe,pressures_allowed,sacks_allowed,Season
0,DEN,10,16,624,25,74,624,90.1,111,12,2014-15
1,CIN,7,16,545,21,71,545,89.4,104,12,2014-15
2,GB,12,16,595,18,76,595,89.1,112,18,2014-15
3,DAL,9,16,517,17,81,517,88.8,107,9,2014-15
4,BLT,3,16,576,21,83,576,88.7,117,13,2014-15
...,...,...,...,...,...,...,...,...,...,...,...
27,NE,19,17,626,27,156,626,79.8,218,35,2023-24
28,CAR,5,17,684,29,174,684,79.8,240,37,2023-24
29,NYG,21,17,665,20,184,665,78.6,244,40,2023-24
30,NYJ,22,17,692,41,171,692,77.9,259,47,2023-24


Feature Selection

In [8]:
pff_receiving_final = pff_receiving[['player', 'player_id', 'Season', 'position', 'team_name', 'player_game_count', 'avg_depth_of_target', 'caught_percent', 'franchise_id', 'grades_offense', 'grades_pass_route', 'pass_plays', 'receptions', 'route_rate', 'routes', 'slot_rate', 'targets', 'touchdowns', 'yards', 'wide_rate', 'inline_rate', 'yards_after_catch', 'yprr']]

pff_man_zone_final = pff_man_zone[['player', 'player_id', 'Season', 'position', 'team_name', 'franchise_id', 'man_grades_pass_route', 'zone_grades_pass_route']]

pfr_fp_final = pfr_fp[['Player', 'Tm', 'Season', 'Age', 'FantPos', 'G', 'GS', 'PPR', 'PosRank', '-9999']]

pfr_fp_prev = pfr_fp[['PPR', 'Season', '-9999', 'G']]

pfr_fp_prev2 = pfr_fp[['PPR', 'Season', '-9999', 'G']]

pfr_rz_final = pfr_rz[['Player', 'Tm', 'Season', 'Tgt20']]

pfr_tm_off_final = pfr_tm_off[['Tm', 'Team_G', 'Season', 'P_Att', 'P_Yds', 'P_TD', 'P_Cmp']]

pff_qb_final = pff_qb[['qb', 'qb_id','Season','grades_pass']]

pff_qb_starters_final = pff_qb_starters[['qb', 'qb_id', 'Team', 'Season']]

pff_ol_final = pff_ol[['team_name', 'Season', 'pbe']]

Preparing data to be merged

In [9]:
# Let's make Team Abbreviation Standard
# ARI | ATL | BAL | BUF | CAR | CHI | CIN | CLE | DAL | DEN | DET | GB | HOU | IND | JAX | KC
# LAC (SD) | LAR (SL) | LV (OAK) | MIA | MIN | NE | NO | NYG | NYJ | PHI | PIT | SEA | SF | TB | TEN | WAS

# PFF -> Standard
pff_team_key = {'ARZ': 'ARI', 'BLT': 'BAL', 'CLV': 'CLE', 'HST': 'HOU', 'LA': 'LAR', 'OAK': 'LV', 'SL': 'LAR', 'SD':'LAC'}
# PFR Abreviation -> Standard
pfr_team_abv_key = {'GNB':'GB', 'KAN':'KC', 'LVR':'LV', 'NWE':'NE', 'NOR':'NO', 'OAK':'LV', 'SDG':'LAC', 'SFO':'SF', 'STL':'LAR', 'TAM':'TB'}
# PFR Full Name -> Standard
pfr_team_name_key = {'Arizona Cardinals':'ARI', 'Atlanta Falcons':'ATL', 'Baltimore Ravens': 'BAL', 'Buffalo Bills': 'BUF', 'Carolina Panthers':'CAR',
                      'Chicago Bears':'CHI', 'Cincinnati Bengals':'CIN', 'Cleveland Browns':'CLE', 'Dallas Cowboys':'DAL', 'Denver Broncos':'DEN', 
                      'Detroit Lions': 'DET', 'Green Bay Packers':'GB', 'Houston Texans':'HOU', 'Indianapolis Colts':'IND', 'Jacksonville Jaguars':'JAX', 
                      'Kansas City Chiefs':'KC', 'Miami Dolphins':'MIA', 'Minnesota Vikings':'MIN', 'New England Patriots':'NE','New Orleans Saints':'NO', 
                      'New York Giants':'NYG', 'New York Jets':'NYJ', 'Oakland Raiders':'LV', 'Las Vegas Raiders':'LV', 'Philadelphia Eagles':'PHI', 
                      'Pittsburgh Steelers':'PIT', 'San Diego Chargers':'LAC', 'Los Angeles Chargers':'LAC', 'San Francisco 49ers':'SF', 'Seattle Seahawks':'SEA', 
                      'St. Louis Rams':'LAR', 'Los Angeles Rams':'LAR', 'Tampa Bay Buccaneers':'TB', 'Tennessee Titans':'TEN', 'Washington Redskins':'WAS', 
                      'Washington Commanders':'WAS', 'Washington Football Team':'WAS'}

for key in pff_team_key:
    pff_receiving_final = pff_receiving_final.replace(key, pff_team_key[key])
    pff_man_zone_final = pff_man_zone_final.replace(key, pff_team_key[key])
    pff_ol_final = pff_ol_final.replace(key, pff_team_key[key])

for key in pfr_team_abv_key:
    pfr_fp_final = pfr_fp_final.replace(key, pfr_team_abv_key[key])
    pfr_rz_final = pfr_rz_final.replace(key, pfr_team_abv_key[key])

for key in pfr_team_name_key:
    pfr_tm_off_final = pfr_tm_off_final.replace(key, pfr_team_name_key[key])


# Some players have different names between different files, which unfortunately is what I have to go off of.
# I am changing the PFR names to reflect the PFF names

name_key = {"John Metchie":"John Metchie III", "Calvin Austin":"Calvin Austin III", "Marvin Mims": "Marvin Mims Jr.", "Easop Winston": "Easop Winston Jr.", 
            "David Sills": "David Sills V", "Josh Palmer": "Joshua Palmer", "James Proche": "James Proche II", "Terry Godwin":"Terry Godwin II", "DeMichael Harris": "De'Michael Harris",
            "Lynn Bowden Jr.": "Lynn Bowden", "Gabriel Davis": "Gabe Davis", "Steven Mitchell": "Steven Mitchell Jr.", "Stanley Morgan Jr.": "Stanley Morgan", 
            "JJ Arcega-Whiteside":"J.J. Arcega-Whiteside", "Mecole Hardman":"Mecole Hardman Jr.", "JJ Jones": "J.J. Jones", "Victor Bolden": "Victor Bolden Jr.",
            "Ray-Ray McCloud": "Ray-Ray McCloud III", "DJ Chark":"DJ Chark Jr.", "D.J. Moore":"DJ Moore", "Jojo Natson":"JoJo Natson", "Keelan Cole":"Keelan Cole Sr.",
            "Jakeem Grant":"Jakeem Grant Sr.", "Demarcus Ayers":"De'Marcus Ayers", "Roger Lewis":"Roger Lewis Jr.", "Walt Powell":"Walter Powell", "Will Fuller":"Will Fuller V",
            "Chris Harper":"Christopher Harper", "Terrelle Pryor":"Terrelle Pryor Sr.", "JJ Nelson":"J.J. Nelson", "Donteea Dye":"Donteea Dye Jr.", "Rashad Greene":"Rashad Greene Sr.",
            "Steve Smith Sr.":"Steve Smith", "Marvin Jones":"Marvin Jones Jr.", "Willie Snead":"Willie Snead IV", "Josh Cribbs":"Joshua Cribbs", "De'Andre Presley": "DeAndre Presley", 
            "Paul Richardson":"Paul Richardson Jr.", "Stevie Johnson": "Steve Johnson", "Louis Murphy": "Louis Murphy Jr.", "Allen Robinson": "Allen Robinson II", 
            "Odell Beckham": "Odell Beckham Jr.", "Michael Pittman": "Michael Pittman Jr."}
for key in name_key:
    pfr_fp_final = pfr_fp_final.replace(key, name_key[key])
    pfr_rz_final = pfr_rz_final.replace(key, name_key[key])

# In PFR, some people are noted as being on 2TM or 3TM or 4TM. Here we will take a subset of those players,
# and then merge them later to the data without using team as a key

pfr_rz_mult_tms = pfr_rz_final[pfr_rz_final['Tm'].isin(['2TM', '3TM', '4TM'])]

print(pfr_tm_off_final)

     Tm  Team_G   Season  P_Att   P_Yds  P_TD  P_Cmp
0    GB    16.0  2014-15  536.0  4261.0  38.0  349.0
1   DEN    16.0  2014-15  607.0  4661.0  40.0  399.0
2   PHI    16.0  2014-15  621.0  4356.0  27.0  384.0
3    NE    16.0  2014-15  609.0  4121.0  34.0  392.0
4   DAL    16.0  2014-15  476.0  3784.0  37.0  328.0
..  ...     ...      ...    ...     ...   ...    ...
27  PIT    17.0  2023-24  506.0  3163.0  13.0  323.0
28  NYJ    17.0  2023-24  601.0  2919.0  11.0  356.0
29  NYG    17.0  2023-24  518.0  2886.0  15.0  338.0
30   NE    17.0  2023-24  557.0  3069.0  16.0  351.0
31  CAR    17.0  2023-24  586.0  2741.0  13.0  350.0

[320 rows x 7 columns]


Joining tables together

In [10]:
# Data Model
# pfr_fp: Fantasy Points from Current Year
# pff_receiving: Receiving Statistics from Previous Year
# pff_man_zone: Man/Zone Statistics from Previous Year
# pfr_rz: Red Zone Statistics from Previous Year
# pfr_tm_off: Current Year team's Offensive Statistics from Previous Year
# pfr_qb: Current Year team's pfr_qb_starter Statistics from Previous Year
# pfr_ol: Current Year team's OL Statistics from Previous Year

# Start off with pfr_fp_final without including the oldest year in the data

df = pfr_fp_final[pfr_fp_final['Season'] != '2014-15']

# Create column for prev_season

season_key = {'2023-24': '2022-23', '2022-23':'2021-22', '2021-22':'2020-21', '2020-21':'2019-20', 
              '2019-20':'2018-19', '2018-19':'2017-18', '2017-18':'2016-17', '2016-17':'2015-16', 
              '2015-16':'2014-15'}

df['prev_season'] = df['Season'].map(season_key)

# Create column for prev_2_season

season_key = {'2023-24': '2021-22', '2022-23':'2020-21', '2021-22':'2019-18', '2020-21':'2018-19', 
              '2019-20':'2017-18', '2018-19':'2016-17', '2017-18':'2015-16', '2016-17':'2014-15'}

df['prev_2_season'] = df['Season'].map(season_key)

# Joining df to previous season's pfr_fp_prev to get last season's PPR points

pfr_fp_prev['PPR/G'] = pfr_fp_prev['PPR'] / pfr_fp_prev['G']
pfr_fp_prev = pfr_fp_prev.rename(columns={"PPR/G":"PPR/G_prev", "G":"G_prev"})
pfr_fp_prev = pfr_fp_prev[['PPR/G_prev', 'Season', '-9999']]

df = pd.merge(left=df, right=pfr_fp_prev, how='left', left_on=['-9999', 'prev_season'], right_on=['-9999', 'Season'], suffixes=[None, '_copy'])
df = df.drop(columns=['Season_copy'])

# Joining df to 2 season's ago pfr_fp_prev2 to get PPR points from 2 season's ago

pfr_fp_prev2['PPR/G'] = pfr_fp_prev2['PPR'] / pfr_fp_prev2['G']
pfr_fp_prev2 = pfr_fp_prev2.rename(columns={"PPR/G":"PPR/G_prev2"})
pfr_fp_prev2 = pfr_fp_prev2[['PPR/G_prev2', 'Season', '-9999']]

df = pd.merge(left=df, right=pfr_fp_prev2, how='left', left_on=['-9999', 'prev_2_season'], right_on=['-9999', 'Season'], suffixes=[None, '_copy'])
df = df.drop(columns=['Season_copy'])

# Joining df with current season pff_receiving just to get player_id for future joins. This will only work
# for players on multiple teams.

player_ids = pff_receiving_final[['player', 'player_id', 'Season', 'team_name']]
df = pd.merge(left=df, right=player_ids, how='left', left_on=['Player', 'Tm', 'Season'], right_on=['player', 'team_name', 'Season'], suffixes=[None, '_copy'])
df = df.drop(columns=['player', 'team_name'])

# Join df with current season pff_receiving data without using teams for players that were on multiple teams.
# This will fill in the player_id information for players that were on multiple teams and will do nothing for
# all other players.

df = pd.merge(left=df, right=player_ids, how='left', left_on=['Player', 'Season'], right_on=['player', 'Season'], suffixes=[None, '_copy'])
df['player_id'].fillna(df['player_id_copy'], inplace=True)
df = df.drop(columns=['player_id_copy', 'player', 'team_name'])

# Joining with with pff_receiving data to get previous year's receiving data

df = pd.merge(left=df, right=pff_receiving_final, how='left', left_on=['player_id', 'prev_season'], right_on=['player_id', 'Season'], suffixes=[None,'_copy'])
df = df.drop(columns=['Season_copy'])
df = df.rename(columns={'team_name':'prev_team'})

# Joining with pff_man_zone to get previous year's statistics against man and zone coverage

df = pd.merge(left=df, right=pff_man_zone_final, how='left', left_on=['player_id', 'prev_season'], right_on=['player_id', 'Season'], suffixes=[None, '_copy'])
df = df.drop(columns=['player_copy', 'Season_copy', 'position_copy', 'franchise_id_copy'])

# Joining with pfr_rz data to get previous year's redzone data

df = pd.merge(left=df, right=pfr_rz_final, how='left', left_on=['Player', 'prev_team', 'prev_season'], right_on=['Player', 'Tm', 'Season'], suffixes=[None, '_copy'])
df = df.drop(columns=['Season_copy', 'Tm_copy'])

# Joining with pfr_rz_mult_tms to get previous year's redzone data for players on multiple teams

df = pd.merge(left=df, right=pfr_rz_mult_tms, how='left', left_on=['Player', 'prev_season'], right_on=['Player', 'Season'], suffixes=[None, '_copy'])
df['Tgt20'].fillna(df['Tgt20_copy'], inplace=True)
df = df.drop(columns=['Tm_copy', 'Season_copy', 'Tgt20_copy'])

# Joining with pff_qb_starters to find the starter you need

df = pd.merge(left=df, right=pff_qb_starters_final, how='left', left_on=['Tm', 'Season'], right_on=['Team', 'Season'], suffixes=[None, '_copy'])
df = df.drop(columns=['Team'])

# Joining with pff_qb to get the grade of the current qb in the previous season

df = pd.merge(left=df, right=pff_qb_final, how='left', left_on=['qb_id', 'prev_season'], right_on=['qb_id', 'Season'], suffixes=[None, '_copy'])
df = df.drop(columns=['qb_copy', 'Season_copy'])

# Joining with pff_ol to get the grade of the current team offensive line's pass blocking in the previous season

df = pd.merge(left=df, right=pff_ol_final, how='left', left_on=['Tm', 'prev_season'], right_on=['team_name', 'Season'], suffixes=[None, '_copy'])

# Joining with pfr_tm_off to get statistics about the current team's offense from previous season

df = pd.merge(left=df, right=pfr_tm_off_final, how='left', left_on=['Tm', 'prev_season'], right_on=['Tm', 'Season'], suffixes=[None, '_copy'])
df = df.drop(columns=['Season_copy'])

df.to_csv('merged_df.csv')

C:\Users\scott\AppData\Local\Temp\ipykernel_17892\242620775.py:20: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['prev_season'] = df['Season'].map(season_key)
C:\Users\scott\AppData\Local\Temp\ipykernel_17892\242620775.py:27: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['prev_2_season'] = df['Season'].map(season_key)
C:\Users\scott\AppData\Local\Temp\ipykernel_17892\242620775.py:31: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,

Calculate Variable to determine whether or not the WR is on an upward trend

In [11]:
df['Improvement'] = (df['PPR/G_prev'] - df['PPR/G_prev2']) / df['PPR/G_prev2']

Calculate Variable to determine how many targets were vacated in this team



Create variable to determine Depth Chart place

Turn Metrics into Per Game

In [12]:

df['PPR/G'] = df['PPR'] /df['G']
df['Pass Plays/G'] = df['pass_plays'] / df['player_game_count']
df['Receptions/G'] = df['receptions'] / df['player_game_count']
df['Routes/G'] = df['routes'] / df['player_game_count']
df['Targets/G'] = df['targets'] / df['player_game_count']
df['TD/G'] = df['touchdowns'] / df['player_game_count']
df['YDS/G'] = df['yards'] / df['player_game_count']
df['YAC/G'] = df['yards_after_catch'] / df['player_game_count']
df['RZ Targets/G'] = df['Tgt20'] / df['player_game_count']
df['P_Att/G'] = df['P_Att'] / df['Team_G']
df['P_Cmp/G'] = df['P_Cmp'] / df['Team_G']
df['P_TD/G'] = df['P_TD'] / df['Team_G']
df['P_Yds/G'] = df['P_Yds'] / df['Team_G']



df = df.drop(columns=['PPR', 'pass_plays', 'receptions', 'routes', 'targets', 'touchdowns', 'yards', 
                      'yards_after_catch', 'Tgt20', 'P_Att', 'P_Cmp', 'P_TD', 'P_Yds'])



Drop columns that were needed for joins but aren't needed anymore

In [13]:
df = df.drop(columns=['FantPos', 'G', 'GS', '-9999', 'prev_season', 'prev_2_season', 'player_id', 'player', 'position', 
                      'team_name', 'player_game_count', 'franchise_id', 'qb', 'qb_id', 'Team_G', 'prev_team', 'team_name_copy'])

# Temporary dropping to limit na's

df = df.drop(columns=['Improvement', 'PPR/G_prev2'])

df.to_csv('cleaned_df.csv')

for col in df.columns:
    print(col + ": " + str(df[col].isnull().sum() / len(df[col])))

nonull = df.dropna()
nonull.to_csv('no_null_df.csv')

Player: 0.0
Tm: 0.0
Season: 0.0
Age: 0.0
PosRank: 0.0
PPR/G_prev: 0.3350045578851413
avg_depth_of_target: 0.3796718322698268
caught_percent: 0.3796718322698268
grades_offense: 0.3796718322698268
grades_pass_route: 0.3796718322698268
route_rate: 0.3796718322698268
slot_rate: 0.3796718322698268
wide_rate: 0.3796718322698268
inline_rate: 0.3796718322698268
yprr: 0.3796718322698268
man_grades_pass_route: 0.3814949863263446
zone_grades_pass_route: 0.3810391978122151
grades_pass: 0.11713764813126709
pbe: 0.0414767547857794
PPR/G: 0.11941659070191431
Pass Plays/G: 0.3796718322698268
Receptions/G: 0.3796718322698268
Routes/G: 0.3796718322698268
Targets/G: 0.3796718322698268
TD/G: 0.3796718322698268
YDS/G: 0.3796718322698268
YAC/G: 0.3796718322698268
RZ Targets/G: 0.47219690063810393
P_Att/G: 0.0414767547857794
P_Cmp/G: 0.0414767547857794
P_TD/G: 0.0414767547857794
P_Yds/G: 0.0414767547857794


In [14]:
# STILL NEED TO DO:

# Depth Chart place
# Vacated Target Share
# Improvement

In [15]:
##df = pff_receiving[pff_receiving['Season'] == '2023-24']
#df[['player', 'PPR PPG']].sort_values(['PPR PPG'], ascending=False).head(20)